# Placeholder Notebook

Fill in code & analysis here.

In [ ]:
import random
from gensim.models import Word2Vec

# 1. 加载模型和数据
model = Word2Vec.load("../outputs/cbow_model")
sentences = [l.split() for l in open("../data/clean/comments.txt", encoding="utf-8")]

# 2. 定义 Top-k 精度函数
def topk_acc(k, trials=1000):
    correct = 0
    total = 0
    for _ in range(trials):
        sent = random.choice(sentences)
        # 先过滤出在 vocab 里的单词
        sent = [w for w in sent if w in model.wv.key_to_index]
        if len(sent) < 3:
            continue  # 太短或全是 OOV，跳过
        idx = random.randrange(1, len(sent)-1)
        target = sent[idx]
        context = sent[:idx] + sent[idx+1:]
        # 再次检查：context 必须非空
        if not context:
            continue
        # 计算所有词的平均相似度
        sims = {}
        for w in model.wv.key_to_index:  # 遍历词表里的候选
            # 平均上下文相似度
            score = sum(model.wv.similarity(w, c) for c in context) / len(context)
            sims[w] = score
        # 取 top-k
        topk = sorted(sims, key=sims.get, reverse=True)[:k]

        total += 1
        if target in topk:
            correct += 1

    return correct / total if total > 0 else 0

# 3. 输出结果
print("▶ Start eval…")
print("Top-1:", topk_acc(1, trials=50))
print("Top-3:", topk_acc(3, trials=50))


▶ Start eval…
